In [7]:
import pandas as pd
from pandas_datareader.data import DataReader
from datetime import datetime
from pathlib import Path


def fetch_fedfunds(start: str, end: str) -> pd.DataFrame:
    """Fetch monthly Effective Federal Funds Rate (FEDFUNDS) from FRED."""
    start_dt = pd.to_datetime(start)
    end_dt = pd.to_datetime(end)
    df = DataReader("FEDFUNDS", "fred", start_dt, end_dt)
    df = df.rename(columns={"FEDFUNDS": "fedfunds"})
    df.index.name = "date"

    # Debug prints
    print(f"\n📊 Retrieved FEDFUNDS data from {start} to {end}")
    print(f"Rows: {len(df)}, Columns: {list(df.columns)}")
    print("First 5 rows:")
    print(df.head())
    print("Last 5 rows:")
    print(df.tail())
    return df


def to_quarterly(df: pd.DataFrame, method: str = "mean") -> pd.DataFrame:
    """Convert monthly FEDFUNDS to quarterly frequency.

    method: 'mean' (quarterly average) or 'last' (end-of-quarter)
    """
    if method == "last":
        q = df.resample("Q").last()
    else:
        q = df.resample("Q").mean()
    q.index.name = "quarter"

    # Debug prints
    print(f"\n📈 Converted to quarterly data using method='{method}'")
    print(f"Rows: {len(q)}, Columns: {list(q.columns)}")
    print("First 5 rows:")
    print(q.head())
    print("Last 5 rows:")
    print(q.tail())
    return q
  
start = "2010-01-01"
end = datetime.today().strftime("%Y-%m-%d")

monthly = fetch_fedfunds(start, end)
quarterly = to_quarterly(monthly, method="mean")



📊 Retrieved FEDFUNDS data from 2010-01-01 to 2025-08-25
Rows: 187, Columns: ['fedfunds']
First 5 rows:
            fedfunds
date                
2010-01-01      0.11
2010-02-01      0.13
2010-03-01      0.16
2010-04-01      0.20
2010-05-01      0.20
Last 5 rows:
            fedfunds
date                
2025-03-01      4.33
2025-04-01      4.33
2025-05-01      4.33
2025-06-01      4.33
2025-07-01      4.33

📈 Converted to quarterly data using method='mean'
Rows: 63, Columns: ['fedfunds']
First 5 rows:
            fedfunds
quarter             
2010-03-31  0.133333
2010-06-30  0.193333
2010-09-30  0.186667
2010-12-31  0.186667
2011-03-31  0.156667
Last 5 rows:
            fedfunds
quarter             
2024-09-30  5.263333
2024-12-31  4.650000
2025-03-31  4.330000
2025-06-30  4.330000
2025-09-30  4.330000
